In [1]:
from pro.data.vocdata import VocDataset
label_dir = 'pro/data/dataset/VOCdevkit/VOC2007/Annotations'
image_dir = 'pro/data/dataset/VOCdevkit/VOC2007/JPEGImages'
voc_dataset = VocDataset(label_dir, image_dir)
print('datanumber: {}'.format(voc_dataset.__len__()))
# next(iter(voc_dataset))


datanumber: 5011


In [3]:
from pro.data.vocdata import VocDataloader
voc_dataloader= VocDataloader(voc_dataset)
train_dataloader, val_dataloader = voc_dataloader(0.7,batch_size_train=3,sample=100)

print('batch number of train_data: {},val_data: {}'.format(train_dataloader.__len__(),val_dataloader.__len__()))

batch number of train_data: 12,val_data: 16


In [4]:
from pro.model.model import FasterRcnn
from pro.util.utils import get_class_dict

class_name_dict = get_class_dict(label_dir)
print(class_name_dict)
fasterrcnn_cls = FasterRcnn(class_name_dict, device='cpu')

{0: 'diningtable', 1: 'chair', 2: 'horse', 3: 'person', 4: 'tvmonitor', 5: 'bird', 6: 'cow', 7: 'dog', 8: 'bottle', 9: 'pottedplant', 10: 'aeroplane', 11: 'car', 12: 'cat', 13: 'sheep', 14: 'bicycle', 15: 'sofa', 16: 'boat', 17: 'train', 18: 'motorbike', 19: 'bus'}


In [7]:
# fasterrcnn_cls.train(train_dataloader, num_epochs=1)

In [5]:
fasterrcnn_cls.eval(val_dataloader, 
            load_from='output_model/221216-175225/train_ALL_VOC2007.cpu.pt', 
            plot_result=True)

100%|██████████| 16/16 [00:19<00:00,  1.24s/it]


mAP over 16 images: 0.000
images output in /Users/hayashi/Documents/code/20221130_fastrcnn/faster-rcnn-lin-2.0/detect/test


TypeError: cannot unpack non-iterable NoneType object

In [11]:
from pro.util.utils import plot_results
plot_results()

In [13]:
from torchmetrics.detection.mean_ap import MeanAveragePrecision
metric  = MeanAveragePrecision()
metric.update(pred, target_gt)
mAP = metric.compute()['map']

In [14]:
mAP


tensor(0.)

In [16]:
pred[1]

{'boxes': tensor([[432.,  86., 460., 147.],
         [225.,  62., 399., 274.],
         [ 79.,  93., 117., 129.],
         [450.,  63., 497., 233.],
         [265., 132., 289., 156.],
         [  5.,  32.,  84., 146.],
         [ 45.,  15., 400., 332.],
         [176.,  38., 367., 330.],
         [  6.,  11., 230., 308.],
         [341.,  79., 397., 250.]], dtype=torch.float64),
 'scores': tensor([0.9956, 0.9925, 0.9891, 0.9863, 0.9786, 0.9769, 0.9698, 0.8764, 0.7695,
         0.5728], dtype=torch.float64),
 'labels': tensor([4., 4., 4., 4., 4., 4., 4., 4., 4., 4.], dtype=torch.float64)}

In [15]:
target_gt[1]

{'boxes': tensor([[384., 199., 415., 269.],
         [448.,  69., 500., 239.],
         [235.,  61., 437., 333.],
         [  1.,   5., 466., 333.],
         [  1.,  34., 105., 155.],
         [433.,  91., 458., 146.],
         [344., 117., 390., 208.],
         [366., 100., 403., 135.],
         [ 86.,  90., 117., 131.],
         [245., 106., 268., 146.],
         [265., 138., 287., 157.],
         [206., 107., 270., 165.]], dtype=torch.float64),
 'labels': tensor([8., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.], dtype=torch.float64)}